#### Import the aima-python repo and any necessary libraries

In [1]:
import sys
import os

# Add the path to the aima-python repo to sys.path
sys.path.append(os.path.abspath('./aima_python'))

from aima_python.games4e import Game
from aima_python.utils4e import vector_add, MCT_Node, ucb
from collections import defaultdict

import random
random.seed(109)

#### Implement our verison of Mancala by subclassing the Game class from aima

In [2]:
class Mancala(Game):
    """Mancala game implementation."""

    def __init__(self):
        self.squares = {i for i in range(14) if i != 6 and i != 13}  # Indices of pits (excluding mancalas)
        self.initial = Board(pits=[4] * 6 + [0] + [4] * 6 + [0], to_move='P1')

    def actions(self, board):
        """Return a list of legal moves (non-empty pits on the current player's side)."""
        start, end = (0, 6) if board.to_move == 'P1' else (7, 13)
        return [i for i in range(start, end) if board.pits[i] > 0]

    def result(self, board, action):
        """Apply the move (distributing stones) and return the new board state."""
        pits = board.pits[:]
        player = board.to_move
        stones = pits[action]
        pits[action] = 0

        idx = action
        while stones > 0:
            idx = (idx + 1) % 14
            if (player == 'P1' and idx == 13) or (player == 'P2' and idx == 6):
                continue  # Skip opponent's mancala
            pits[idx] += 1
            stones -= 1

        # Capture condition
        if (player == 'P1' and 0 <= idx < 6 or player == 'P2' and 7 <= idx < 13) \
                and pits[idx] == 1 and pits[12 - idx] > 0:
            pits[6 if player == 'P1' else 13] += pits[idx] + pits[12 - idx]
            pits[idx] = pits[12 - idx] = 0

        # Determine next player
        next_to_move = player if (player == 'P1' and idx == 6 or player == 'P2' and idx == 13) else ('P2' if player == 'P1' else 'P1')

        return board.new({'pits': pits, 'to_move': next_to_move})

    def utility(self, board, player):
        """Return the game utility for the given player."""
        if self.is_terminal(board): # Calculate the utility function only at terminal nodes
            score_p1 = sum(board.pits[:7])  # P1's Mancala score
            score_p2 = sum(board.pits[7:])  # P2's Mancala score
            if player == 'P1':
                return score_p1 - score_p2  # Max - Min for P1
            else:
                return score_p2 - score_p1  # Max - Min for P2
            return 0

    def is_terminal(self, board):
        """Check if the game has ended."""
        return all(p == 0 for p in board.pits[:6]) or all(p == 0 for p in board.pits[7:13])

    def display(self, board, current_player=None, move_from=None):
        """Display the board state."""
        
        print("  " + " ".join(map(str, board.pits[12:6:-1])))
        print(f"{board.pits[13]}                  {board.pits[6]}")
        print("  " + " ".join(map(str, board.pits[:6])))
        print("\n")

        if current_player is not None and move_from is not None:
            print(f"Player {current_player} moved from pit {move_from}")

#### Implement our version of the mancala board

In [3]:
class Board(defaultdict):
    """A Mancala board with pits and a player to move."""

    def __init__(self, pits=None, to_move=None, **kwds):
        super().__init__(int)
        self.pits = pits or [0] * 14
        self.to_move = to_move

    def new(self, changes: dict, **kwds) -> 'Board':
        """Create a new board state with the specified changes."""
        board = Board(pits=self.pits[:], to_move=self.to_move, **kwds)
        board.__dict__.update(changes)
        return board

    def __hash__(self):
        return hash(tuple(self.pits)) + hash(self.to_move)

    def __repr__(self):
        return f"Mancala({self.pits}, {self.to_move})"

#### Create a random player and a play game function to be able to play games

In [4]:
# Example game simulation
from random import choice

def random_player(game, board):
    """
    A strategy function for a player that selects moves randomly.

    Parameters:
    - game: The Mancala game instance.
    - board: The current game state.

    Returns:
    - A randomly selected legal move from the current game state.
    """
    # Get the list of all legal moves for the current player.
    legal_moves = game.actions(board)
    # Randomly choose one move from the list of legal moves.
    return choice(legal_moves)

def play_game(game, players, verbose=False):
    """
    Simulates a full game of Mancala between two players.

    Parameters:
    - game: The Mancala game instance.
    - players: A dictionary mapping player names ('P1', 'P2') to their strategy functions.
    - verbose: Boolean flag to enable or disable detailed game state output.

    Returns:
    - The utility value of the game for Player 1.
    """
    # Initialize the game state to the starting configuration.
    state = game.initial
    while not game.is_terminal(state):  # Continue until the game ends.
        # Determine which player's turn it is.
        current_player = players[state.to_move]
        # The current player selects a move.
        action = current_player(game, state)
        # Apply the move and update the game state.
        state = game.result(state, action)
        
        if verbose:  # If verbose mode is enabled, display the current game state.
            game.display(state)

    # Return the utility value of the final game state for Player 1.
    return game.utility(state, 'P1')

#### Use this function to set what type of player is playing

In [5]:
def player(search_algorithm):
    """
    Wraps a search algorithm into a strategy function that can be used by the game simulation.

    Parameters:
    - search_algorithm: The algorithm (e.g., minimax, alpha-beta) that determines the player's moves.

    Returns:
    - A strategy function that uses the specified search algorithm to decide the next move.
    """
    # The returned function takes the game instance and the current state as inputs
    # and returns the move (action) suggested by the search algorithm.
    return lambda game, state: search_algorithm(game, state)

#### Use the simulate game function in order to simulate numerous games
#### We need to update this so we can choose the minimax or alphabeta players

In [6]:
def simulate_game(player1, player2, start_with_p1=True):
    """
    Simulates a single game of Mancala between two specified players.

    Parameters:
    - player1: Strategy function for Player 1 (e.g., minimax, alpha-beta, random).
    - player2: Strategy function for Player 2 (e.g., minimax, alpha-beta, random).
    - start_with_p1: Boolean indicating whether Player 1 starts the game.

    Returns:
    - winner: The winner of the game ('P1' or 'P2').
    - moves: The total number of moves made during the game.
    """
    # Create a new Mancala game instance.
    game = Mancala()
    # Initialize the game state and set the starting player.
    board = game.initial.new({'to_move': 'P1' if start_with_p1 else 'P2'})
    moves = 0  # Initialize the move counter.

    while not game.is_terminal(board):  # Continue until the game ends.
        # Determine the current player based on whose turn it is.
        current_player = player1 if board.to_move == 'P1' else player2
        # The current player selects a move.
        action = current_player(game, board)
        # Apply the move and update the board state.
        board = game.result(board, action)
        moves += 1  # Increment the move counter.

    # Determine the winner based on the scores in the mancalas.
    winner = 'P1' if sum(board.pits[:7]) > sum(board.pits[7:]) else 'P2'
    return winner, moves

In [7]:
def simulate_multiple_games(player1, player2, num_games=100):
    """
    Simulates multiple Mancala games between two players and computes statistics.

    Parameters:
    - player1: Strategy function for Player 1 (e.g., minimax, alpha-beta, random).
    - player2: Strategy function for Player 2 (e.g., minimax, alpha-beta, random).
    - num_games: Number of games to simulate.

    Returns:
    - p1_win_percentage: Percentage of games won by Player 1.
    - p2_win_percentage: Percentage of games won by Player 2.
    - avg_moves: Average number of moves per game.
    """
    # Initialize counters for wins and total moves.
    p1_wins, p2_wins, total_moves = 0, 0, 0

    for i in range(num_games):  # Simulate the specified number of games.
        # Alternate the starting player for fairness.
        start_with_p1 = (i % 2 == 0)
        # Simulate a single game.
        winner, moves = simulate_game(player1, player2, start_with_p1)
        # Update the win counters based on the winner.
        if winner == 'P1':
            p1_wins += 1
        else:
            p2_wins += 1
        # Update the total move counter.
        total_moves += moves

    # Calculate win percentages for both players.
    p1_win_percentage = (p1_wins / num_games) * 100
    p2_win_percentage = (p2_wins / num_games) * 100
    # Calculate the average number of moves per game.
    avg_moves = total_moves / num_games

    # Return the computed statistics.
    return p1_win_percentage, p2_win_percentage, avg_moves

In [8]:
def minimax(game, state, max_depth=5):
    """
    Implements the Minimax algorithm with depth-limited search to evaluate moves in a two-player game.

    Parameters:
    - game: The Mancala game instance.
    - state: The current state of the game.
    - max_depth: The maximum depth to explore in the game tree (limits recursion for efficiency).

    Returns:
    - The best action (move) for the current player.
    """
    def max_value(state, depth):
        """
        Evaluate the maximum possible value for the 'maximizing player' (Player 1).

        Parameters:
        - state: Current state of the game.
        - depth: Remaining depth to search in the game tree.

        Returns:
        - The maximum utility value that can be achieved from this state.
        """
        # Base case: terminal state or depth limit reached.
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state)
        
        v = -float('inf')  # Initialize with negative infinity for maximization.
        for action in game.actions(state):  # Explore all legal moves.
            # Recursively evaluate the resulting states as the minimizing player.
            v = max(v, min_value(game.result(state, action), depth - 1))
        return v

    def min_value(state, depth):
        """
        Evaluate the minimum possible value for the 'minimizing player' (Player 2).

        Parameters:
        - state: Current state of the game.
        - depth: Remaining depth to search in the game tree.

        Returns:
        - The minimum utility value that can be achieved from this state.
        """
        # Base case: terminal state or depth limit reached.
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state)
        
        v = float('inf')  # Initialize with positive infinity for minimization.
        for action in game.actions(state):  # Explore all legal moves.
            # Recursively evaluate the resulting states as the maximizing player.
            v = min(v, max_value(game.result(state, action), depth - 1))
        return v

    # Main body of minimax: Determine the best action for the current player.
    best_action = None  # Placeholder for the optimal action.
    # Player 1 (maximizing) wants the highest value; Player 2 (minimizing) wants the lowest.
    best_value = -float('inf') if state.to_move == 'P1' else float('inf')

    for action in game.actions(state):  # Explore each legal action.
        if state.to_move == 'P1':  # Maximizing player.
            value = min_value(game.result(state, action), max_depth - 1)
            if value > best_value:
                best_value, best_action = value, action
        else:  # Minimizing player.
            value = max_value(game.result(state, action), max_depth - 1)
            if value < best_value:
                best_value, best_action = value, action

    return best_action  # Return the action that optimizes the player's outcome.

def heuristic_evaluation(state):
    """
    Evaluates the game state heuristically to guide the Minimax or Alpha-Beta search.

    Parameters:
    - state: The current game state.

    Returns:
    - A heuristic score that estimates the value of the state for the maximizing player.
    """
    # Stones in Player 1's Mancala (index 6) and Player 2's Mancala (index 13).
    p1_mancala = state.pits[6]
    p2_mancala = state.pits[13]

    # Stones remaining in the pits for both players.
    stones_on_board_p1 = sum(state.pits[:6])
    stones_on_board_p2 = sum(state.pits[7:13])

    # Extra turn opportunities: Favor states where the player has empty pits
    # that allow capturing stones from the opponent's side.
    extra_turns = sum(1 for i in range(6) if state.pits[i] == 0 and state.pits[12 - i] > 0)

    # Combine factors into a weighted score:
    # - Mancala score difference.
    # - Stones remaining on the board.
    # - Potential for extra turns.
    return (p1_mancala - p2_mancala) + 0.5 * (stones_on_board_p1 - stones_on_board_p2) + 0.3 * extra_turns

In [9]:
def alphabeta(game, state, alpha=-float('inf'), beta=float('inf'), max_depth=5):
    """
    Implements the Alpha-Beta Pruning algorithm to optimize the Minimax search by pruning branches
    that cannot influence the final decision.

    Parameters:
    - game: The Mancala game instance.
    - state: The current state of the game.
    - alpha: The best value achievable by the maximizing player so far.
    - beta: The best value achievable by the minimizing player so far.
    - max_depth: The maximum depth to explore in the game tree.

    Returns:
    - The best action (move) for the current player.
    """
    def max_value(state, alpha, beta, depth):
        """
        Evaluate the maximum possible value for the 'maximizing player' with pruning.

        Parameters:
        - state: Current state of the game.
        - alpha: Current best value for maximizing player.
        - beta: Current best value for minimizing player.
        - depth: Remaining depth to search in the game tree.

        Returns:
        - The maximum utility value and corresponding action for the maximizing player.
        """
        # Base case: terminal state or depth limit reached.
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state), None

        v, best_action = -float('inf'), None
        for action in game.actions(state):  # Explore all legal moves.
            # Recursively evaluate the resulting states as the minimizing player.
            v2, _ = min_value(game.result(state, action), alpha, beta, depth - 1)
            if v2 > v:
                v, best_action = v2, action  # Update the best value and action.
            if v >= beta:  # Prune the branch if value exceeds beta.
                break
            alpha = max(alpha, v)  # Update alpha with the best value so far.
        return v, best_action

    def min_value(state, alpha, beta, depth):
        """
        Evaluate the minimum possible value for the 'minimizing player' with pruning.

        Parameters:
        - state: Current state of the game.
        - alpha: Current best value for maximizing player.
        - beta: Current best value for minimizing player.
        - depth: Remaining depth to search in the game tree.

        Returns:
        - The minimum utility value and corresponding action for the minimizing player.
        """
        # Base case: terminal state or depth limit reached.
        if game.is_terminal(state) or depth == 0:
            return heuristic_evaluation(state), None

        v, best_action = float('inf'), None
        for action in game.actions(state):  # Explore all legal moves.
            # Recursively evaluate the resulting states as the maximizing player.
            v2, _ = max_value(game.result(state, action), alpha, beta, depth - 1)
            if v2 < v:
                v, best_action = v2, action  # Update the best value and action.
            if v <= alpha:  # Prune the branch if value falls below alpha.
                break
            beta = min(beta, v)  # Update beta with the best value so far.
        return v, best_action

    # Start the search as the maximizing player (default).
    return max_value(state, alpha, beta, max_depth)[1]

In [10]:
minimax_player = player(lambda game, state: minimax(game, state))
alphabeta_player = player(lambda game, state: alphabeta(game, state))

In [11]:
p1_win_percentage, p2_win_percentage, avg_moves = simulate_multiple_games(minimax_player, random_player, num_games=100)
print("Minimax Player vs Random Player")
print(f"Player 1 win percentage: {p1_win_percentage}%")
print(f"Player 2 win percentage: {p2_win_percentage}%")
print(f"Average number of moves: {avg_moves}")

Minimax Player vs Random Player
Player 1 win percentage: 97.0%
Player 2 win percentage: 3.0%
Average number of moves: 35.96


In [12]:
p1_win_percentage, p2_win_percentage, avg_moves = simulate_multiple_games(alphabeta_player, random_player, num_games=100)
print("AlphaBeta Player vs Random Player")
print(f"Player 1 win percentage: {p1_win_percentage}%")
print(f"Player 2 win percentage: {p2_win_percentage}%")
print(f"Average number of moves: {avg_moves}")

AlphaBeta Player vs Random Player
Player 1 win percentage: 99.0%
Player 2 win percentage: 1.0%
Average number of moves: 34.79
